In [ ]:
# pip show earthengine-api geemap ipywidgets ipython

# Version 1
- Flood Frequency 
- Flood Pronearea
- Slope Flat area from SRTM
- Montly water surface layer by Time period 

In [ ]:
# import ee
# import geemap
# import datetime
# import ipywidgets as widgets
# from IPython.display import display

# # Initialize Earth Engine API
# ee.Initialize()

# # Create map
# Map = geemap.Map()

# # Define region of interest (ROI) for Guyana
# gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
# guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))
# georgetown = guyana.filter(ee.Filter.eq('ADM1_NAME', 'Georgetown'))
# roi = guyana.geometry()

# # Define dynamic time period
# start_date = datetime.date(1984, 3, 16)
# end_date = datetime.date(2024, 12, 31)

# # Convert to ee.Date objects
# start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
# end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# # Load datasets
# jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
# srtm = ee.Image("USGS/SRTMGL1_003")

# # Filter JRC dataset for the selected time period and region
# jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)

# # Step 1: Exclude permanent water bodies
# permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
# jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))

# # Step 2: Calculate Flood Frequency
# def calculate_flood_frequency(collection):
#     # Detect observations and water occurrences
#     def add_bands(img):
#         obs = img.gt(0).rename("obs")
#         water = img.select("water").eq(2).rename("water")
#         return img.addBands([obs, water])
    
#     collection = collection.map(add_bands)
#     total_obs = collection.select("obs").sum().rename("total_obs")
#     total_water = collection.select("water").sum().rename("total_water")
#     flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
#     return flood_frequency.updateMask(flood_frequency.neq(0))

# flood_frequency = calculate_flood_frequency(jrc_filtered)

# # Step 3: Analyze DEM
# # Calculate slope and flat areas
# slope = ee.Terrain.slope(srtm).rename("slope")
# flat_area = slope.lt(5).rename("flat_area")  
# low_lying = srtm.lt(10).rename("low_lying")  

# # Step 4: Flood-Prone Areas
# flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))

# # Step 5: Monthly Water Surface Maps
# def get_monthly_flood_layers(collection, roi, permanent_water):
#     """
#     Extracts monthly flood layers from the given collection,
#     excluding permanent water bodies.
#     """
#     monthly_layers = {}
#     months = collection.aggregate_array("system:index").getInfo()
    
#     for month in months:
#         img = collection.filter(ee.Filter.eq("system:index", month)).first()
#         water_layer = img.select("water").eq(2).updateMask(permanent_water.Not())
#         monthly_layers[month] = water_layer.clip(roi)
    
#     return monthly_layers

# # Call the function with the permanent water exclusion
# monthly_layers = get_monthly_flood_layers(jrc_filtered, roi, permanent_water)

# # Visualization parameters
# dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
# slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
# flat_area_vis = {"min": 0, "max": 1, "palette": ["ffffff", "0000ff"]}
# flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
# flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
# water_surface_vis = {"min": 0, "max": 1, "palette": ["0000ff"]}

# # Interactive widgets
# start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
# end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
# run_button = widgets.Button(description="Run Analysis", button_style="success")


# def update_map(change):
#     start_date = start_date_picker.value
#     end_date = end_date_picker.value

#     # Convert to ee.Date objects
#     start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
#     end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

#     # Re-filter the JRC dataset with the new date range
#     jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
#     # Exclude permanent water bodies
#     jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
#     # Calculate flood frequency
#     flood_frequency = calculate_flood_frequency(jrc_filtered)
#     # Calculate flood-prone areas
#     flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
#     # flood_prone_area = flood_frequency.updateMask(flat_area)
    
#     # Add new layers without removing existing ones
#     Map.centerObject(roi, 8)
#     Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
#     Map.addLayer(slope.clip(roi), slope_vis, "Slope")
#     Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, f"Flood Frequency from {start_date} to {end_date}")
#     Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, f"Flood-Prone Areas from {start_date} to {end_date}")
   
#     # for month, layer in monthly_layers.items():
#     #     Map.addLayer(layer, water_surface_vis, f"Water Surface {month}")

# # Set the button action
# run_button.on_click(update_map)

# # Display the widgets
# display(start_date_picker, end_date_picker, run_button)

# # Show the map
# Map.addLayerControl()
# Map


# Version 2 
* added low-lying buttom

In [ ]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display
import re  # Import regex module

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Load GAUL dataset and filter for Guyana
gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))

# Get list of available cities in Guyana
raw_city_list = guyana.aggregate_array('ADM1_NAME').getInfo()

# Clean region names by removing "(region N°X)"
city_list = [re.sub(r"\s*\(region N°\d+\)", "", name) for name in raw_city_list]

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")

# Interactive widgets
city_selector = widgets.Dropdown(
    options=city_list,
    value=city_list[0],
    description="City:",
)
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
base_map_selector = widgets.Dropdown(
    options=["SATELLITE", "OSM"],
    value="SATELLITE",
    description="Base Map:",
)
run_button = widgets.Button(description="Run Analysis", button_style="success")

    
def update_map(change):
    selected_city = city_selector.value
    
    # Match the cleaned city name with the original dataset
    raw_city_name = next(name for name in raw_city_list if selected_city in name)
    roi = guyana.filter(ee.Filter.eq('ADM1_NAME', raw_city_name)).geometry()
    
    start_date = start_date_picker.value
    end_date = end_date_picker.value
    low_lying_threshold = low_lying_input.value  # Get updated low-lying level
    selected_basemap = base_map_selector.value  # Get selected base map
    
    # Convert to ee.Date objects
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)
    
    # Filter datasets to ROI
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    
    # Calculate flood frequency
    def calculate_flood_frequency(collection):
        def add_bands(img):
            obs = img.gt(0).rename("obs")
            water = img.select("water").eq(2).rename("water")
            return img.addBands([obs, water])
        
        collection = collection.map(add_bands)
        total_obs = collection.select("obs").sum().rename("total_obs")
        total_water = collection.select("water").sum().rename("total_water")
        flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
        return flood_frequency.updateMask(flood_frequency.neq(0))
    
    flood_frequency = calculate_flood_frequency(jrc_filtered)
    slope = ee.Terrain.slope(srtm).rename("slope")
    flat_area = slope.lt(5).rename("flat_area")
    low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
    # Remove previous layers **safely**
    layer_names = [
        "Flood Frequency",
        "Flood-Prone Areas",
        "SRTM DEM",
        "Slope"
    ]
    for layer_name in layer_names:
        try:
            Map.remove_layer(layer_name)
        except Exception:
            pass  # Ignore errors if the layer is not found
    
    # Set the selected base map
    Map.add_basemap(selected_basemap)
    Map.centerObject(roi, 8)
    
    # Visualization parameters
    dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
    slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
    flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
    flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}

    # Add layers again
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
    Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")


# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# Ensure the base map is added initially
# Map.add_basemap("SATELLITE")  
Map.addLayerControl()
Map

# version 3
* ESA LULC as Layer
* Google Open building as Laver

In [ ]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display
import re  # Import regex module

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Load GAUL dataset and filter for Guyana
gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))

# Get list of available cities in Guyana
raw_city_list = guyana.aggregate_array('ADM1_NAME').getInfo()

# Clean region names by removing "(region N°X)"
city_list = [re.sub(r"\s*\(region N°\d+\)", "", name) for name in raw_city_list]

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")
worldcover = ee.ImageCollection("ESA/WorldCover/v200").select("Map").first()  # Load ESA WorldCover dataset

# Interactive widgets
city_selector = widgets.Dropdown(
    options=city_list,
    value=city_list[0],
    description="City:",
)
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
base_map_selector = widgets.Dropdown(
    options=["SATELLITE", "OSM"],
    value="SATELLITE",
    description="Base Map:",
)
run_button = widgets.Button(description="Run Analysis", button_style="success")

    
def update_map(change):
    selected_city = city_selector.value
    
    # Match the cleaned city name with the original dataset
    raw_city_name = next(name for name in raw_city_list if selected_city in name)
    roi = guyana.filter(ee.Filter.eq('ADM1_NAME', raw_city_name)).geometry()
    
    start_date = start_date_picker.value
    end_date = end_date_picker.value
    low_lying_threshold = low_lying_input.value  # Get updated low-lying level
    selected_basemap = base_map_selector.value  # Get selected base map
    
    # Convert to ee.Date objects
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)
    
    # Filter datasets to ROI
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    
    # Calculate flood frequency
    def calculate_flood_frequency(collection):
        def add_bands(img):
            obs = img.gt(0).rename("obs")
            water = img.select("water").eq(2).rename("water")
            return img.addBands([obs, water])
        
        collection = collection.map(add_bands)
        total_obs = collection.select("obs").sum().rename("total_obs")
        total_water = collection.select("water").sum().rename("total_water")
        flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
        return flood_frequency.updateMask(flood_frequency.neq(0))
    
    flood_frequency = calculate_flood_frequency(jrc_filtered)
    slope = ee.Terrain.slope(srtm).rename("slope")
    flat_area = slope.lt(5).rename("flat_area")
    low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
    # Landcover data visualization
    landcover_roi = worldcover.clip(roi)
    # landcover_vis = {"min": 10, "max": 100, "palette": ["ff0000", "00ff00", "0000ff"]}  # Example palette
    # landcover_vis = {
    # "bands": ["Map"],  # Explicitly specify the 'Map' band
    # }

    landcover_vis = {
        "bands": ["Map"],
        "min": 10,
        "max": 100,
        "palette": [
            "#006400", "#ffbb22", "#ffff4c", "#f096ff", "#fa0000",
            "#b4b4b4", "#f0f0f0", "#0064c8", "#0096a0", "#00cf75", "#fae6a0"
        ]
    }

    # Load Google Open Buildings dataset
    open_buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v3/polygons').filterBounds(roi)
    buildings_vis = {"color": '00FF00'} 

    # Remove previous layers **safely**
    layer_names = [
        "Flood Frequency",
        "Flood-Prone Areas",
        "SRTM DEM",
        "Slope",
        "Land Use (ESA WorldCover)"
    ]
    for layer_name in layer_names:
        try:
            Map.remove_layer(layer_name)
        except Exception:
            pass  # Ignore errors if the layer is not found
    
    # Set the selected base map
    Map.add_basemap(selected_basemap)
    Map.centerObject(roi, 8)
    
    # Visualization parameters
    dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
    slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
    flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
    flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}

    # Add layers again
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(landcover_roi, landcover_vis, "Land Use (ESA WorldCover)")
    Map.addLayer(open_buildings, buildings_vis, "Open Buildings")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
    Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")
    
    add_legend()

def add_legend():
    legend_dict = {
        "Trees": "#006400",
        "Shrubland": "#ffbb22",
        "Grassland": "#ffff4c",
        "Cropland": "#f096ff",
        "Built-up": "#fa0000",
        "Bare / Sparse Veg": "#b4b4b4",
        "Snow & Ice": "#f0f0f0",
        "Permanent Water": "#0064c8",
        "Herbaceous Wetland": "#0096a0",
        "Mangroves": "#00cf75",
        "Moss / Lichen": "#fae6a0",
    }

    Map.add_legend(title="ESA WorldCover 2020", legend_dict=legend_dict)



# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# Ensure the base map is added initially
Map.addLayerControl()
Map


# Version 4
- count flooded building

In [13]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display
import re  # Import regex module

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Load GAUL dataset and filter for Guyana
gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))

# Get list of available cities in Guyana
raw_city_list = guyana.aggregate_array('ADM1_NAME').getInfo()

# Clean region names by removing "(region N°X)"
city_list = [re.sub(r"\s*\(region N°\d+\)", "", name) for name in raw_city_list]

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().select("Map")  # Fixed selection issue
buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")  # Open Buildings dataset

# Interactive widgets
city_selector = widgets.Dropdown(
    options=city_list,
    value=city_list[0],
    description="City:",
)
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
base_map_selector = widgets.Dropdown(
    options=["SATELLITE", "OSM"],
    value="SATELLITE",
    description="Base Map:",
)
run_button = widgets.Button(description="Run Analysis", button_style="success")


def update_map(change):
    selected_city = city_selector.value
    
    # Match the cleaned city name with the original dataset
    raw_city_name = next(name for name in raw_city_list if selected_city in name)
    roi = guyana.filter(ee.Filter.eq('ADM1_NAME', raw_city_name)).geometry()
    
    start_date = start_date_picker.value
    end_date = end_date_picker.value
    low_lying_threshold = low_lying_input.value  # Get updated low-lying level
    selected_basemap = base_map_selector.value  # Get selected base map
    
    # Convert to ee.Date objects
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)
    
    # Filter datasets to ROI
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    
    # Calculate flood frequency
    def calculate_flood_frequency(collection):
        def add_bands(img):
            obs = img.gt(0).rename("obs")
            water = img.select("water").eq(2).rename("water")
            return img.addBands([obs, water])
        
        collection = collection.map(add_bands)
        total_obs = collection.select("obs").sum().rename("total_obs")
        total_water = collection.select("water").sum().rename("total_water")
        flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
        return flood_frequency.updateMask(flood_frequency.neq(0))
    
    flood_frequency = calculate_flood_frequency(jrc_filtered)
    slope = ee.Terrain.slope(srtm).rename("slope")
    flat_area = slope.lt(5).rename("flat_area")
    low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
    # Convert building polygons to centroids
    building_centroids = buildings.filterBounds(roi).map(lambda f: f.centroid())

    # Count flooded buildings
    flooded_buildings = building_centroids.filterBounds(flood_prone_area.geometry())
    flooded_building_count = flooded_buildings.size().getInfo()

    # Landcover visualization
    landcover_roi = worldcover.clip(roi)
    landcover_vis = {
        "bands": ["Map"],
        "min": 10,
        "max": 100,
        "palette": [
            "#006400", "#ffbb22", "#ffff4c", "#f096ff", "#fa0000",
            "#b4b4b4", "#f0f0f0", "#0064c8", "#0096a0", "#00cf75", "#fae6a0"
        ]
    }
    
    # Remove previous layers safely
    layer_names = [
        "Flood Frequency",
        "Flood-Prone Areas",
        "SRTM DEM",
        "Slope",
        "Land Use (ESA WorldCover)",
        "Buildings",
        "Flooded Buildings"
    ]
    for layer_name in layer_names:
        try:
            Map.remove_layer(layer_name)
        except Exception:
            pass  # Ignore errors if the layer is not found
    
    # Set the selected base map
    Map.add_basemap(selected_basemap)
    Map.centerObject(roi, 8)
    
    # Visualization parameters
    dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
    slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
    flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
    flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
    building_vis = {"color": "blue"}
    flooded_building_vis = {"color": "red"}

    # Add layers
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
    Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")
    Map.addLayer(landcover_roi, landcover_vis, "Land Use (ESA WorldCover)")
    Map.addLayer(building_centroids, building_vis, "Buildings")
    Map.addLayer(flooded_buildings, flooded_building_vis, "Flooded Buildings")

    # Add flooded building count as text overlay (legend)
    Map.add_text(
        text=f"Flooded Buildings: {flooded_building_count}",
        position="topright",
        font_size=14,
        text_color="white",
        background_color="black",
        name="Flooded Buildings Count"
    )


    add_legend()

def add_legend():
    legend_dict = {
        "Trees": "#006400",
        "Shrubland": "#ffbb22",
        "Grassland": "#ffff4c",
        "Cropland": "#f096ff",
        "Built-up": "#fa0000",
        "Bare / Sparse Veg": "#b4b4b4",
        "Snow & Ice": "#f0f0f0",
        "Permanent Water": "#0064c8",
        "Herbaceous Wetland": "#0096a0",
        "Mangroves": "#00cf75",
        "Moss / Lichen": "#fae6a0",
    }

    Map.add_legend(title="ESA WorldCover 2020", legend_dict=legend_dict)


# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# Ensure the base map is added initially
Map.addLayerControl()
Map


Dropdown(description='City:', options=('Barima Waini', 'Cuyuni/mazaruni', 'Demerara Mahaica', 'East Berbice/co…

DatePicker(value=datetime.date(1984, 3, 16), description='Start Date', step=1)

DatePicker(value=datetime.date(2024, 12, 31), description='End Date', step=1)

BoundedFloatText(value=10.0, description='Low-Lying Level (m)', step=1.0)

Dropdown(description='Base Map:', options=('SATELLITE', 'OSM'), value='SATELLITE')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…